In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Imports

In [ ]:
import cv2
import pickle
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os
import tensorflow as tf
import importlib

from keras.initializers import glorot_uniform


root_path = '/content/drive/MyDrive/CVDL'

dumps_path =  f"{root_path}/dumps/"
dataset  = f"{root_path}/dataset/" 

!cp "$root_path/modules/lib.py" .
import lib

!cp "$root_path/modules/plots.py" .
import plots

!cp "$root_path/modules/metrics.py" .
import metrics




# Parameters

In [ ]:
# Deafault Res: 606x468 (ar 101 : 78)

# Tested Res's:
# 224x160 (ar 7 : 5)

resx = 384
resy = 288

seed = 42

params = {
          'x':resx,                                
          'y':resy,                                
          'batch_size': 16,                        
          'n_channels_mask': 1,                            
          'n_channels': 1,                        
          'shuffle': True,              
          'learningRate': 0.002,                      
          'momentum': 0.98,                        
          'nEpoches': 20,                        
          }



ratio_test = 0.8
ratio_val  = 0.9

lib.create_dump([params, seed], dumps_path + "params_du")


type_ds  = ['Images','Masks']
modality = ['train','validation','test']

# get path Img and Mask
path_img  = os.path.join(dataset, type_ds[0])
path_msk  = os.path.join(dataset, type_ds[1])
path_     = [path_img, path_msk]

# get length imgs and proof that is equal with masks
length_imgs  = len(os.listdir(os.path.join(dataset,type_ds[0] + "/"))) 
length_masks = len(os.listdir(os.path.join(dataset,type_ds[1] + "/")))
params['length_data'] = length_imgs,

# get name of file imgs
imgs_list_path = os.listdir(path_img)

# Data loading 

In [ ]:
# LOAD & SPLIT DATA ============================================================
mode = 1  # mode = 0: Random Split
          # mode = 1: Split for Patients
          
dataset_list = lib.get_dataset_list(
                                mode, 
                                path_, 
                                imgs_list_path, 
                                params, 
                                ratio_test, 
                                ratio_val, 
                                seed, 
                                modality)

# Dumping splitted data
lib.create_dump(dataset_list, f"{dumps_path}splitted_data_{resx}x{resy}")

#Loading Data from Pickle

In [ ]:
dataset_list = pickle.load(open( f"{dumps_path}splitted_data_{resx}x{resy}", "rb" ))

#Data Augmentation

In [ ]:
# DATA AUGMENTATION ============================================================
train_generator, val_generator = lib.data_augmentation(seed, dataset_list, params)

params["length_training"]   = len(dataset_list[0]['train'])
params['length_validation'] = len(dataset_list[0]['validation'])

augmented_batch_plot = plots.plot_augmented_batch(train_generator)
plt.show()

# Model definition & training

In [ ]:
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint

# -- Available archs --
# unet
# doubleunet
arch_name = 'doubleunet'
!cp "/content/drive/MyDrive/Colab Notebooks/CVDL/modules/models/{arch_name}.py" .
arch = importlib.import_module(arch_name)

input_size = (resy, resx, 1)

initializer = glorot_uniform() # -> unet/ lightweight only
model = arch.build_model(input_size)
model.summary()

In [ ]:
# Default loss, metric
#loss = 'binary_crossentropy'
#metric = 'accuracy'

loss = metrics.dice_loss
metric = metrics.dice

train_history, model = lib.train(root_path, params, model, loss, metric, train_generator, val_generator)

train_figure = plots.plot_train_metric(train_history, params['nEpoches'], "dice")

plt.show()

In [ ]:
#Dumping model weights
model.save('/content/drive/MyDrive/CVDL/Dataset/results/models/unet_lr001_e100_bs14')

# Testing results

In [ ]:
# Set the threshold for predictions
threshold = 0.4

results_df, avgs = plots.get_results(dataset_list, params, model, threshold)
print(results_df)
print(avgs)

In [ ]:
res = results_df.loc[(results_df['dice'] < 0.60)]
print(res)

num = res['dice'].count()
den = results_df['dice'].count()

print(num/den)

In [ ]:
# Set the threshold for predictions
threshold = 0.4

# Select the image on which make 
image_index = 18
test_image = cv2.resize(dataset_list[0]['test'][image_index,:,:,0],(params["x"],params["y"]))
test_mask = cv2.resize(dataset_list[1]['test'][image_index,:,:,0],(params["x"],params["y"]))

test_results = lib.get_preditcions(test_image, test_mask, params, model, threshold)

print("testing accuracy:", lib.accuracy(test_mask, test_results["pred_mask"]))
print("testing dice:", lib.dice(test_mask, test_results["pred_mask"]))

results_figure = plots.plot_test_results(test_results)
plt.show()